### 사전 준비 사항 

#### (1) uv add (터미널)

```bash
uv add pdfplumber sentence-transformers faiss-cpu numpy torch python-dotenv openai
```

#### (2) .env 파일 세팅
```bash
OPENAI_API_KEY = ""
HF_TOKEN = ""
```

In [93]:
import pdfplumber
import os
from pathlib import Path
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

In [94]:
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
client = OpenAI()
MODEL = "gpt-4o-mini"

In [95]:
import json

BASE_DIR = Path.cwd().parent  # /codeit-part3-team4
RAW_FOLDER = BASE_DIR / "data/raw/files"  # 수정된 경로

# 폴더에서 PDF 목록 가져오기
def get_pdf_paths(folder_path: Path | str) -> list[Path]:
    folder = Path(folder_path)
    pdf_paths = [p for p in folder.glob("*.pdf")]
    return sorted(pdf_paths)

PDF_PATH_LIST = get_pdf_paths(RAW_FOLDER)


all_data_path = os.path.join(BASE_DIR, "data", "ALL_DATA.json")

with open(all_data_path, "r", encoding="utf-8") as f:
    ALL_DATA = json.load(f)

In [96]:
# PDF to text
def extract_text(pdf_path: Path | str) -> str:
    texts = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            texts.append(page.extract_text() or "")
    return "\n".join(texts)

In [97]:
# Chunking
def chunk(text: str, size: int = 800) -> list[str]:
    return [text[i:i+size] for i in range(0, len(text), size)]

In [98]:
# 임베딩 및 인덱스 만들기
def build_index(chunks: list[str]) -> tuple:
    model = SentenceTransformer("all-MiniLM-L6-v2")
    embs = model.encode(chunks)

    index = faiss.IndexFlatL2(embs.shape[1])
    index.add(np.array(embs).astype("float32"))
    return index, model, chunks

In [99]:
# RFP 분석용 프롬프트
RFP_PROMPT = """
너는 정부·공공기관 제안요청서(RFP)를 분석하는 전문가다.
아래 컨텍스트는 하나의 정부 RFP 문서에서 추출된 내용이다.

다음 기준에 따라 **핵심 정보만 구조적으로 요약**해라.

[요약 규칙]
- 추측하지 말고, 문서에 명시된 내용만 사용
- 불필요한 설명, 배경, 미사여구 제거
- 사업 참여 판단에 필요한 정보 위주로 요약
- 항목이 없으면 "명시 없음"으로 표기

[출력 형식]
아래 각 항목은 **질문에 답하는 형태**로 작성하되,
반드시 `key: 답변` 형식으로 출력하라.
(예시) project_name: 고려대학교 차세대 포털·학사 정보시스템 구축 사업

project_name: 사업(용역)명은 무엇인가?
agency: 발주 기관(수요기관)은 어디인가?
purpose: 사업 목적(추진 배경)은 무엇인가?
budget: 총 사업 예산(사업비)은 얼마인가?
base_amount: 추정 가격/기초금액은 얼마인가?
contract_type: 계약 방식(일반경쟁/제한경쟁/협상에 의한 계약 등)은 무엇인가?
deadline: 입찰/제안서 제출 마감일시는 언제인가?
submission_method: 제출 장소 또는 제출 방법(온라인/방문/우편)은 무엇인가?
submission_docs: 제출 서류 목록은 무엇인가?
format_requirement: 제안서(기술/가격) 제출 파일 형식이나 양식은 무엇인가?
duration: 사업 수행 기간은 얼마나 되는가?
start_date: 사업 착수(시작) 예정일은 언제인가?
end_date: 사업 종료(납품) 예정일은 언제인가?
scope: 제안 요청 범위(Scope/업무 범위)는 무엇인가?
requirements_must: 필수 요구사항(기능/성능/보안 등)은 무엇인가?
requirements_optional: 선택 요구사항(우대/가점)에는 무엇이 있는가?
eval_items: 평가 항목(기술/가격 등) 구성은 어떻게 되는가?
eval_weights: 평가 배점(점수) 비율은 어떻게 되는가?
price_eval: 가격 평가 방식(최저가/협상 등)은 무엇인가?
presentation: 제안 발표(PT/발표평가)가 있는가? 있다면 일정은?
qa_period: 질의응답(Q&A) 접수 기간은 언제인가?
qa_method: 질의응답 제출 방법/양식은 무엇인가?
briefing: 현장설명회가 있는가? 있다면 일정/장소는?
eligibility: 참가 자격 요건(면허/실적/인증/등급)은 무엇인가?
consortium: 공동수급/하도급/컨소시엄 가능 여부는?
staffing: 필수 인력 요건(PM/보안담당 등)이 있는가?
security: 보안/개인정보 요구사항(인증, ISMS 등)은 무엇인가?
deliverables: 납품물(산출물) 목록은 무엇인가?
maintenance: 유지보수/운영 조건(기간/범위)은 무엇인가?
contact: 문의처(담당자/전화/이메일)는 무엇인가?

[컨텍스트]
{context}

이제 위 형식에 맞춰 한국어로 요약해라.
"""

In [100]:
# 쿼리 답변
def answer(index, model, chunks, query: str, top_k: int = 15) -> str:
    q_emb = model.encode([query])
    _, I = index.search(np.array(q_emb).astype("float32"), top_k)
    
    context = "\n\n".join(chunks[i] for i in I[0])
    prompt = RFP_PROMPT.format(context=context)
    
    resp = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "RFP 전문 분석가. 형식 엄수."},
            {"role": "user", "content": prompt},
        ],
        max_tokens=2000,
    )
    return resp.choices[0].message.content

In [101]:
# 데이터 폴더 설정
docs = get_pdf_paths(RAW_FOLDER)
print(f"발견된 PDF: {len(docs)}개")
for i, doc in enumerate(docs):
    print(f"{i}: {doc.name}")

발견된 PDF: 100개
0: (사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf
1: (사)부산국제영화제_2024년 BIFF & ACFM 온라인서비스 재개발 및 행사지원시.pdf
2: (사）한국대학스포츠협의회_KUSF 체육특기자 경기기록 관리시스템 개발.pdf
3: (재)예술경영지원센터_통합 정보시스템 구축 사전 컨설팅.pdf
4: 2025 구미 아시아육상경기선수권대회 조직위원회_2025 구미아시아육상경.pdf
5: BioIN_의료기기산업 종합정보시스템(정보관리기관) 기능개선 사업(2차).pdf
6: KOICA 전자조달_[긴급] [지문] [국제] 우즈베키스탄 열린 의정활동 상하원 .pdf
7: 경기도 안양시_호계체육관 배드민턴장 및 탁구장 예약시스템 구축 용역.pdf
8: 경기도 평택시_2024년도 평택시 버스정보시스템(BIS) 구축사업.pdf
9: 경기도사회서비스원_2024년 통합사회정보시스템 운영지원.pdf
10: 경상북도 봉화군_봉화군 재난통합관리시스템 고도화 사업(협상)(긴급).pdf
11: 경희대학교_[입찰공고] 산학협력단 정보시스

In [102]:
def format_table(table: dict) -> str:
    """table_content를 읽기 좋은 텍스트로 변환."""
    tc = table.get("table_content", {})
    cols = tc.get("columns", [])
    rows = tc.get("data", [])
    if not cols and not rows:
        return ""
    
    lines = []
    title = table.get("table_title", "")
    if title:
        lines.append(f"[표] {title}")
    lines.append(" | ".join(str(c or "") for c in cols))
    for row in rows:
        lines.append(" | ".join(str(c or "") for c in row))
    return "\n".join(lines)

In [103]:
import re

def clean_text(text: str) -> str:
    """PDF 추출 텍스트에서 노이즈 제거."""
    # 페이지 번호 (- 1 -, - 23 - 등)
    text = re.sub(r'^- \d+ -\n?', '', text.strip())
    # 목차 점선 (···, ……, .... 등)
    text = re.sub(r'[·.…]{5,}', ' ', text)
    # 같은 글자 5회 이상 반복 (목   목   목... / 차차차차...)
    text = re.sub(r'(.)\1{4,}', '', text)
    # 연속 공백 → 단일 공백
    text = re.sub(r' {2,}', ' ', text)
    # 연속 빈줄 → 단일 빈줄
    text = re.sub(r'\n{3,}', '\n\n', text)
    return text.strip()

In [104]:
def chunk_from_alldata(doc_name: str, all_data: dict, size: int = 800) -> list[str] | None:
    """ALL_DATA의 섹션 단위로 그룹핑 후 size 단위로 분할."""
    if doc_name not in all_data:
        return None

    pages = all_data[doc_name]["metadata"]

    # 1) 섹션별로 페이지 텍스트 그룹핑
    sections = []  # [(섹션명, 텍스트)]
    current_sec = None
    current_parts = []

    for page in pages:
        sec = page.get("section")
        if isinstance(sec, list):
            sec_str = " | ".join(sec)
        elif sec:
            sec_str = sec
        else:
            sec_str = ""

        # 섹션이 바뀌면 이전 섹션 저장
        if sec_str != current_sec and current_parts:
            sections.append((current_sec, "\n".join(current_parts)))
            current_parts = []
        current_sec = sec_str

        # 페이지 텍스트 조립
        parts = []
        if sec_str:
            parts.append(f"[섹션: {sec_str}] (p.{page['page'] + 1})")
        else:
            parts.append(f"(p.{page['page'] + 1})")

        if page.get("text"):
            parts.append(clean_text(page["text"]))

        if page.get("table"):
            for t in page["table"]:
                table_text = format_table(t)
                if table_text:
                    parts.append(table_text)

        current_parts.append("\n".join(parts))

    # 마지막 섹션
    if current_parts:
        sections.append((current_sec, "\n".join(current_parts)))

    # 2) 섹션별로 size 단위 분할 (섹션 경계에서 끊김 방지)
    chunks = []
    for sec_name, sec_text in sections:
        for i in range(0, len(sec_text), size):
            chunks.append(sec_text[i:i + size])

    return chunks

In [105]:
# 전체 문서별 인덱스 저장
doc_indexes = {}

for doc_path in docs:
    print(f"처리 중: {doc_path.name}")
    chunks = chunk_from_alldata(doc_path.name, ALL_DATA)

    if chunks is None:
        text = clean_text(extract_text(doc_path))
        chunks = chunk(text)

    index, model, chunks = build_index(chunks)
    doc_indexes[doc_path] = (index, model, chunks)

print("모든 문서 인덱싱 완료!")

처리 중: (사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf


Loading weights: 100%|██████████| 103/103 [00:00<00:00, 1498.53it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


처리 중: (사)부산국제영화제_2024년 BIFF & ACFM 온라인서비스 재개발 및 행사지원시.pdf


Loading weights: 100%|██████████| 103/103 [00:00<00:00, 2663.77it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


처리 중: (사）한국대학스포츠협의회_KUSF 체육특기자 경기기록 관리시스템 개발.pdf


...

처리 중: 서울시립대학교_[사전공개] 학업성취도 다차원 종단분석 통합시스템 1차.pdf


Loading weights: 100%|██████████| 103/103 [00:00<00:00, 2470.38it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


처리 중: 서울특별시_2024년 지도정보 플랫폼 및 전문활용 연계 시스템 고도화 용.pdf


Loading weights: 100%|██████████| 103/103 [00:00<00:00, 2574.17it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


모든 문서 인덱싱 완료!


In [106]:
# 특정 문서의 청크 몇 개를 뽑아서 확인
test_name = str(docs[0]).split("/")[-1]
chunks_test = chunk_from_alldata(test_name, ALL_DATA)

print(f"문서: {test_name}")
print(f"총 청크 수: {len(chunks_test)}")
print()

# 섹션/페이지/테이블이 포함된 청크 찾기
for i, c in enumerate(chunks_test):
    has_section = "[섹션:" in c
    has_page = "(p." in c
    has_table = "[표]" in c
    if has_section or has_table:
        print(f"=== 청크 {i} ({len(c)}자) ===")
        print(c)
        print("...\n")
        if i > 5:
            break

문서: (사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf
총 청크 수: 252

=== 청크 1 (800자) ===
[섹션: 1. 추진개요] (p.3)
Ⅰ. 추진개요1추진배경 및 방향□ 「벤처기업육성에 관한 특별조치법」(이하 ‘벤처기업법’) 복수의결권주식, 스톡옵션(주식매수선택권), 성과조건부주식교부계약(RS) 등의 기능 고도화 및 이관, 신규 구축업무를 本 과업에서 추진 ⚬(복수의결권주식*) 벤처기업법 제16조의11에 따라 발행된 복수의결권주식 보고 업무처리 시스템 구축* 복수의결권이란? 모든 주주는 1주당 하나의 의결권을 갖는 주주평등원칙(상법)과 별도로 비상장 벤처기업 창업자에게만 1주당 최대 10배의 의결권 행사를 부여하는 제도 ⚬(스톡옵션*) 벤처기업법 제16조의3에 따라 부여된 벤처기업 스톡옵션 부여, 취소·철회 신고 및 업무시스템 구축* 스톡옵션이란? 비상장 벤처기업 임·직원 및 기업 성장에 기여한 자에게 미리 정한 가격(행사가격)으로 신주를 인수하거나 자기의 주식을 매수할 수 있는 권리 혹은 주식의 시가와 행사가격의 차액을 청구할 수 있는 권리를 부여하는 제도 -중소벤처24 內 스톡옵션 신청 기능 이관을 추진하며, 과거 데이터 정제 및 적재 등을 통한 데이터 이관 추진 ⚬(성과조건부주식*) 벤처기업법 제16조의19에 따라 성과조건부주식교부계약의 신고 및 업무처리 시스템 구축* 성과조건부주식이란? 비상장 벤처기업 임ㆍ직원 및 특정인에게 자기주식 부여를 약속하되, 일정 재직기간 및 조건을 충족해야 주식이 부여 대상자에게 귀속되도록 제한을 둔 주식□ 복수의결권주식 발행 보고 및 스톡옵션 신고기능은 기존 시스템 내 벤처확인 이력정보와 신고자(벤처기업)의 정보와 연동
[섹션: 1. 추진개요] (p.4)
2사업개요□ (
...

=== 청크 3 (800자) ===
[섹션: 2. 추진방안] (p.5)
Ⅱ. 사업 추진방안1 추진목표추진목표◈ 벤처확인 연계

In [107]:
# 테스트 1: 단일 문서 (docs[0])
if docs:
    test_doc = docs[0]
    print(f"\n=== {test_doc.name} 분석 ===")
    index, model, chunks = doc_indexes[test_doc]
    result = answer(index, model, chunks, "RFP 분석해줘")
    print(result)


=== (사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf 분석 ===
project_name: 사업(용역)명은 무엇인가?  
agency: 발주 기관(수요기관)은 어디인가?  
purpose: 사업 목적(추진 배경)은 무엇인가?  
budget: 총 사업 예산(사업비)은 얼마인가?  
base_amount: 추정 가격/기초금액은 얼마인가?  
contract_type: 계약 방식(일반경쟁/제한경쟁/협상에 의한 계약 등)은 무엇인가?  
deadline: 입찰/제안서 제출 마감일시는 언제인가?  
submission_method: 제출 장소 또는 제출 방법(온라인/방문/우편)은 무엇인가?  
submission_docs: 제출 서류 목록은 무엇인가?  
format_requirement: 제안서(기술/가격) 제출 파일 형식이나 양식은 무엇인가?  
duration: 사업 수행 기간은 얼마나 되는가?  
start_date: 사업 착수(시작) 예정일은 언제인가?  
end_date: 사업 종료(납품) 예정일은 언제인가?  
scope: 제안 요청 범위(Scope/업무 범위)는 무엇인가?  
requirements_must: 필수 요구사항(기능/성능/보안 등)은 무엇인가?  
requirements_optional: 선택 요구사항(우대/가점)에는 무엇이 있는가?  
eval_items: 평가 항목(기술/가격 등) 구성은 어떻게 되는가?  
eval_weights: 평가 배점(점수) 비율은 어떻게 되는가?  
price_eval: 가격 평가 방식(최저가/협상 등)은 무엇인가?  
presentation: 제안 발표(PT/발표평가)가 있는가? 있다면 일정은?  
qa_period: 질의응답(Q&A) 접수 기간은 언제인가?  
qa_method: 질의응답 제출 방법/양식은 무엇인가?  
briefing: 현장설명회가 있는가? 있다면 일정/장소는?  
eligibi

In [108]:
# 테스트 2: 전체 문서 루프 (비용 주의! 주석 해제 후 사용)
# 실행 완료시 rfp_summaries.json 파일이 생성됩니다.

results = {}
for doc_path in docs:
    print(f"\n=== {doc_path.name} 분석 ===")
    index, model, chunks = doc_indexes[doc_path]
    result = answer(index, model, chunks, "RFP 분석해줘")
    results[doc_path.name] = result
    print(result)

# JSON 저장
import json
with open(BASE_DIR / "rfp_summaries_meta_v3.json", "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)
print("전체 분석 완료!")


=== (사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf 분석 ===
project_name: 사업(용역)명은 무엇인가? 명시 없음  
agency: 발주 기관(수요기관)은 어디인가? 명시 없음  
purpose: 사업 목적(추진 배경)은 무엇인가? 운영 안정화  
budget: 총 사업 예산(사업비)은 얼마인가? 명시 없음  
base_amount: 추정 가격/기초금액은 얼마인가? 명시 없음  
contract_type: 계약 방식(일반경쟁/제한경쟁/협상에 의한 계약 등)은 무엇인가? 명시 없음  
deadline: 입찰/제안서 제출 마감일시는 언제인가? 명시 없음  
submission_method: 제출 장소 또는 제출 방법(온라인/방문/우편)은 무엇인가? 명시 없음  
submission_docs: 제출 서류 목록은 무엇인가? 명시 없음  
format_requirement: 제안서(기술/가격) 제출 파일 형식이나 양식은 무엇인가? 명시 없음  
duration: 사업 수행 기간은 얼마나 되는가? 명시 없음  
start_date: 사업 착수(시작) 예정일은 언제인가? 명시 없음  
end_date: 사업 종료(납품) 예정일은 언제인가? 명시 없음  
scope: 제안 요청 범위(Scope/업무 범위)는 무엇인가? 요구분석/설계, 시스템 구축, 사용자 교육, 시스템 오픈  
requirements_must: 필수 요구사항(기능/성능/보안 등)은 무엇인가? 다양한 브라우저 환경 지원, 접근성 준수, 보안 요구사항  
requirements_optional: 선택 요구사항(우대/가점)에는 무엇이 있는가? 명시 없음  
eval_items: 평가 항목(기술/가격 등) 구성은 어떻게 되는가? 명시 없음  
eval_weights: 평가 배점(점수) 비율은 어떻게 되는가? 명시 없음  
price_eval: 가격 평가 방식(최저가/협상 등)은 무엇인가? 